# NLP 2 - Lab 07
## Adrien Giget, Tanguy Malandain, Denis Stojiljkovic

### Imports & Data

In [16]:
import transformers
import torch
import numpy as np
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import evaluate

In [2]:
# Load the IMDB dataset
dataset = load_dataset('imdb')

  0%|          | 0/3 [00:00<?, ?it/s]

### Model

In [36]:
# Choose the model
checkpoint = "distilbert-base-uncased"

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classi

In [4]:
# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [5]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

## 1. (3 points) Fine-tune the model on the training data.

In [6]:
# Define the training arguments
training_args = TrainingArguments(
    "test_trainer",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
)

In [7]:
# Define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
)

The chosen parameters dictate a training process where the model is evaluated after each complete pass (epoch) over the training dataset. The learning rate and weight decay are set to typical values for Transformer models, providing a balance between training speed and model stability. The batch size is selected to manage computational resources effectively. The datasets provided to the trainer have been tokenized to be suitable for the model.

In [8]:
# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.222900,0.194862


TrainOutput(global_step=1563, training_loss=0.2585421493247161, metrics={'train_runtime': 1737.8992, 'train_samples_per_second': 14.385, 'train_steps_per_second': 0.899, 'total_flos': 3311684966400000.0, 'train_loss': 0.2585421493247161, 'epoch': 1.0})

In [9]:
# Check if the model is on GPU
print(model.device) # should print 'cuda' if on GPU

cuda:0


## 2. **\[Bonus\]** Fine-tune your model using the accuracy as evaluation instead of the loss (default).

In [17]:
# Load the accuracy metric from HuggingFace's datasets module
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

In [18]:
# Define the trainer
trainer_accuracy_eval = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,  # Add the compute_metrics function here
)

Advantages:
- Simplicity and Direct Interpretation: Accuracy provides a direct quantification of the proportion of instances correctly classified by the model.
- Effectiveness in Balanced Datasets: In scenarios where classes are equitably represented, accuracy can effectively reflect model performance.

Disadvantages:
- Ineffectiveness in Imbalanced Datasets: Accuracy fails to deliver reliable performance insights in the presence of class imbalance. This is due to its inability to distinguish between the model's capacity to identify minority and majority classes.
- Lack of Discrimination Among Error Types: Accuracy equally penalizes all errors, disregarding the model's confidence level associated with each prediction.
- Absence of Gradual Reward System: Accuracy does not provide any partial credits. In multi-class scenarios, it does not acknowledge near-correct classifications.

In [19]:
# Train the model
trainer_accuracy_eval.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.191400,0.222480,0.927280


TrainOutput(global_step=1563, training_loss=0.1695261532422906, metrics={'train_runtime': 1745.4719, 'train_samples_per_second': 14.323, 'train_steps_per_second': 0.895, 'total_flos': 3311684966400000.0, 'train_loss': 0.1695261532422906, 'epoch': 1.0})

## 3. (2 points) Evaluate the model in term of accuracy on the test data.

In [20]:
# Evaluate the model
eval_results = trainer.evaluate()

# Print the evaluation results
print(eval_results)

{'eval_loss': 0.22247962653636932, 'eval_runtime': 467.207, 'eval_samples_per_second': 53.509, 'eval_steps_per_second': 3.345, 'epoch': 1.0}


In [21]:
# Evaluate the model
eval_acc_results = trainer_accuracy_eval.evaluate()

# Print the evaluation results
print(eval_acc_results)

{'eval_loss': 0.22247962653636932, 'eval_accuracy': 0.92728, 'eval_runtime': 469.5925, 'eval_samples_per_second': 53.238, 'eval_steps_per_second': 3.328, 'epoch': 1.0}


 The high accuracy score and the small loos score suggests that the model correctly predicted the class of the input in about 92.73% of the test samples.

This is a strong result, indicating that the model has learned to make effective predictions from the training data.

## 4. (2 points) For at least 2 samples which have been wrongly classified in the test set, try explaining why the model could have been wrong.

In [22]:
# Get the predictions
predictions, _, _ = trainer.predict(tokenized_datasets["test"])

# The predictions are in a one-hot format. Get the label id with the highest value
predicted_label_ids = np.argmax(predictions, axis=-1)

# Get the actual label ids
actual_label_ids = tokenized_datasets["test"]["label"]

# Find the indices of the wrongly classified samples
wrong_indices = [i for i, (predicted, actual) in enumerate(zip(predicted_label_ids, actual_label_ids)) if predicted != actual]

In [26]:
wrongly_classified_texts = []
# Print two wrongly classified samples
for i in wrong_indices[:2]:
    print(f"Text: {tokenized_datasets['test']['text'][i]}")
    print(f"Predicted: {predicted_label_ids[i]}, Actual: {actual_label_ids[i]}")
    wrongly_classified_texts.append([tokenized_datasets['test']['text'][i],
                                     predicted_label_ids[i],
                                     actual_label_ids[i]])

Text: First off let me say, If you haven't enjoyed a Van Damme movie since bloodsport, you probably will not like this movie. Most of these movies may not have the best plots or best actors but I enjoy these kinds of movies for what they are. This movie is much better than any of the movies the other action guys (Segal and Dolph) have thought about putting out the past few years. Van Damme is good in the movie, the movie is only worth watching to Van Damme fans. It is not as good as Wake of Death (which i highly recommend to anyone of likes Van Damme) or In hell but, in my opinion it's worth watching. It has the same type of feel to it as Nowhere to Run. Good fun stuff!
Predicted: 1, Actual: 0
Text: Ben, (Rupert Grint), is a deeply unhappy adolescent, the son of his unhappily married parents. His father, (Nicholas Farrell), is a vicar and his mother, (Laura Linney), is ... well, let's just say she's a somewhat hypocritical soldier in Jesus' army. It's only when he takes a summer job as

Texte 1:
"First off let me say, If you haven't enjoyed a Van Damme movie since bloodsport, you probably will not like this movie. Most of these movies may not have the best plots or best actors but I enjoy these kinds of movies for what they are. This movie is much better than any of the movies the other action guys (Segal and Dolph) have thought about putting out the past few years. Van Damme is good in the movie, the movie is only worth watching to Van Damme fans. It is not as good as Wake of Death (which i highly recommend to anyone of likes Van Damme) or In hell but, in my opinion it's worth watching. It has the same type of feel to it as Nowhere to Run. Good fun stuff!"

Prediction: 1, Actuel: 0

Ce texte est un cas complexe. L'auteur utilise un langage relativement positif pour décrire le film ("Van Damme est bon dans ce film", "il vaut la peine d'être vu", "C'est un bon film !") et recommande même un autre film du même acteur. Cependant, l'auteur indique également que ce film n'est pas aussi bon que d'autres films de Van Damme, et note qu'il ne vaut la peine d'être vu que par les fans de l'acteur. La critique commence également par un sentiment négatif ("Si vous n'avez pas aimé un film de Van Damme depuis Bloodsport, vous n'aimerez probablement pas ce film"). Ce sentiment mitigé aurait pu empêcher le modèle de classer correctement la critique.

Texte 2:
"Ben, (Rupert Grint), is a deeply unhappy adolescent, the son of his unhappily married parents. His father, (Nicholas Farrell), is a vicar and his mother, (Laura Linney), is ... well, let's just say she's a somewhat hypocritical soldier in Jesus' army. It's only when he takes a summer job as an assistant to a foul-mouthed, eccentric, once-famous and now-forgotten actress Evie Walton, (Julie Walters), that he finally finds himself in true 'Harold and Maude' fashion. Of course, Evie is deeply unhappy herself and it's only when these two sad sacks find each other that they can put their mutual misery aside and hit the road to happiness.<br /><br />Of course it's corny and sentimental and very predictable but it has a hard side to it, too and Walters, who could sleep-walk her way through this sort of thing if she wanted, is excellent. It's when she puts the craziness to one side and finds the pathos in the character, (like hitting the bottle and throwing up in the sink), that she's at her best. The problem is she's the only interesting character in the film (and it's not because of the script which doesn't do anybody any favours). Grint, on the other hand, isn't just unhappy; he's a bit of a bore as well while Linney's starched bitch is completely one-dimensional. (Still, she's got the English accent off pat). The best that can be said for it is that it's mildly enjoyable - with the emphasis on the mildly."

Prediction: 1, Actuel: 0

Cette critique est également assez mitigée. L'auteur utilise des termes positifs pour décrire le film ("Walters, qui pourrait traverser ce genre de choses en somnambule si elle le voulait, est excellente", "elle est à son meilleur", "c'est légèrement agréable") mais critique également d'autres aspects du film (le décrivant comme "ringard, sentimental et très prévisible", "Grint, quant à lui, n'est pas seulement malheureux, il est un peu ennuyeux", "la garce amidonnée de Linney est complètement unidimensionnelle"). Ce mélange de sentiments positifs et négatifs aurait pu perturber le modèle.

Ces deux exemples illustrent un problème courant dans l'analyse des sentiments : les critiques contiennent souvent un mélange de sentiments positifs et négatifs, et il peut être difficile de déterminer le sentiment général de la critique. Les Transformers, comme d'autres modèles d'apprentissage automatique, font des prédictions basées sur des modèles qu'ils ont appris à partir des données d'apprentissage. Si les données d'apprentissage ne contiennent pas suffisamment d'exemples d'avis mitigés similaires, ou si le modèle n'a pas appris à interpréter correctement ces cas complexes, il risque de commettre des erreurs.

En outre, l'analyse des sentiments est une tâche difficile sur laquelle même les humains peuvent être en désaccord. Par exemple, différentes personnes peuvent interpréter différemment le sentiment général de ces avis. Certains peuvent les considérer comme plus positifs parce qu'ils contiennent des éléments positifs, tandis que d'autres peuvent se concentrer davantage sur les éléments négatifs. Il n'est donc pas surprenant qu'un modèle d'apprentissage automatique puisse également éprouver des difficultés dans de tels cas.

## 5. (3 points) What are the advantages and inconvenient of using this model in production compared to the naive Bayes we implemented in the first part of the course? And compared to a recurrent model like an RNN or an LSTM?

Avantages de Transformer (DistilBERT) par rapport à Naive Bayes :

- Meilleures performances : Les transformers sont généralement plus performants que les modèles d'apprentissage automatique traditionnels tels que Naive Bayes dans les tâches de classification de texte. En effet, ils sont capables de modéliser des modèles complexes dans les données et de saisir le contexte d'une manière que Naive Bayes ne peut pas faire.

- Intégrations contextuels : Les transformers génèrent des enchâssements contextuels de mots, ce qui signifie qu'un même mot peut avoir des représentations différentes en fonction de son contexte. Il s'agit d'un avantage significatif par rapport à des méthodes telles que les sacs de mots utilisés dans Naive Bayes, qui traitent chaque occurrence d'un mot comme étant la même, quel que soit le contexte.

- Apprentissage par transfert : Les transformers sont généralement pré-entraînés sur de grandes quantités de données, et ce pré-entraînement peut être exploité pour des tâches spécifiques avec une plus petite quantité de données spécifiques à la tâche. Cela permet d'économiser beaucoup de temps et de ressources informatiques par rapport à la formation d'un modèle à partir de zéro.

Inconvénients de Transformer (DistilBERT) par rapport à Naive Bayes :

- Exigences informatiques : Les transformers sont des modèles beaucoup plus grands et nécessitent plus de ressources informatiques (mémoire, puissance de traitement) que Naive Bayes. Il peut donc être plus difficile de les former et de les utiliser dans un environnement de production, en particulier si les ressources sont limitées.

- Interprétabilité : Les Transformers, comme d'autres modèles d'apprentissage profond, sont souvent considérés comme des modèles "boîte noire", ce qui signifie qu'il est difficile de comprendre pourquoi ils font les prédictions qu'ils font. Naive Bayes, en revanche, est un modèle plus simple et plus facile à interpréter.

Avantages du transformer (DistilBERT) par rapport aux RNNs/LSTMs :

- Parallélisation : Les transformers peuvent traiter simultanément toutes les données d'entrée, ce qui les rend plus rapides et plus efficaces en termes de calcul que les RNN, qui traitent les données de manière séquentielle.

- Traitement des dépendances à long terme : Bien que les LSTM soient conçues pour atténuer le problème du gradient qui s'évanouit et capturer les dépendances à long terme, dans la pratique, elles peuvent encore éprouver des difficultés à cet égard pour les séquences très longues. Les transformateurs utilisent des mécanismes d'auto-attention qui leur permettent de capturer les dépendances quelle que soit la distance dans la séquence d'entrée.

Inconvénients du transformer (DistilBERT) par rapport aux RNN/LSTM :

- Consommation de ressources : Les transformers ont généralement plus de paramètres que les RNN/LSTM, ce qui les rend plus gourmands en ressources.

- Complexité : Les transformers sont des modèles plus complexes, ce qui peut les rendre plus difficiles à mettre en œuvre et à optimiser par rapport à des modèles plus simples comme les RNN/LSTM.

En conclusion, le choix du modèle dépend des exigences spécifiques de votre application, y compris la quantité de données disponibles, les ressources informatiques, la complexité de la tâche et le besoin d'interprétabilité.

## 7. **\[Bonus\]** The model only accepts inputs of maximum 512 tokens. Propose and implement a solution that goes around that.

In [40]:
def sliding_window_inference(text, model, tokenizer, stride=64, max_length=512):
    # Tokenize the text
    tokenized_text = tokenizer.encode(text, truncation=False)

    # Get the number of tokens
    num_tokens = len(tokenized_text)
    
    # Initialize the list to store the predictions
    predictions = []
    
    # Slide the window over the tokenized text
    for i in range(0, num_tokens, stride):
        # Get the end index of the tokens for the current window
        end_index = min(i + max_length, num_tokens)
        
        # Extract the tokens for the current window
        tokens = tokenized_text[i:end_index]
        
        # Convert the tokens to input tensors
        input_tensors = torch.tensor([tokens]).to(model.device)
        
        # Perform inference with the model
        with torch.no_grad():
            outputs = model(input_tensors)
        
        # Get the probabilities of the positive class
        proba = torch.softmax(outputs.logits, dim=-1)[:, 1].item()
        
        # Append the probability to the list of predictions
        predictions.append(proba)
    
    # Return the average prediction
    if predictions:
        return 1 if sum(predictions) / len(predictions) >= 0.5 else 0
    else:
        return 0


In [42]:
long_review_text = ""

# Iterate over the reviews in the training set
for i, example in enumerate(dataset["train"]):
    # Tokenize the review and count the number of tokens
    num_tokens = len(tokenizer.encode(example["text"], truncation=False))
    
    # If the review has more than 512 tokens, print it and break the loop
    if num_tokens > 512:
        long_review_text = example["text"]
        trainer.predict(tokenized_datasets["test"])
        break

Token indices sequence length is longer than the specified maximum sequence length for this model (720 > 512). Running this sequence through the model will result in indexing errors


In [43]:
# Perform sliding window inference on the long review text
prediction = sliding_window_inference(long_review_text, model, tokenizer)
print("Prediction:", prediction)

Prediction: 1


Notre modèle arrive donc bien à traiter des textes de plus de 512 tokens.